In [1]:
from sklearn import model_selection, datasets
from sklearn.metrics import mean_squared_error
from pandas import DataFrame
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_boston

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
data = load_boston()
data_frame = DataFrame(data.data)
data_frame.columns = data.feature_names
data_frame['target'] = data.target

In [4]:
data.data[:,0]

array([  6.32000000e-03,   2.73100000e-02,   2.72900000e-02,
         3.23700000e-02,   6.90500000e-02,   2.98500000e-02,
         8.82900000e-02,   1.44550000e-01,   2.11240000e-01,
         1.70040000e-01,   2.24890000e-01,   1.17470000e-01,
         9.37800000e-02,   6.29760000e-01,   6.37960000e-01,
         6.27390000e-01,   1.05393000e+00,   7.84200000e-01,
         8.02710000e-01,   7.25800000e-01,   1.25179000e+00,
         8.52040000e-01,   1.23247000e+00,   9.88430000e-01,
         7.50260000e-01,   8.40540000e-01,   6.71910000e-01,
         9.55770000e-01,   7.72990000e-01,   1.00245000e+00,
         1.13081000e+00,   1.35472000e+00,   1.38799000e+00,
         1.15172000e+00,   1.61282000e+00,   6.41700000e-02,
         9.74400000e-02,   8.01400000e-02,   1.75050000e-01,
         2.76300000e-02,   3.35900000e-02,   1.27440000e-01,
         1.41500000e-01,   1.59360000e-01,   1.22690000e-01,
         1.71420000e-01,   1.88360000e-01,   2.29270000e-01,
         2.53870000e-01,

In [5]:
data_frame.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [6]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(data.data, data.target, test_size = 0.25)


In [7]:
def H(X,left,right):
    return float(len(left)) / len(X) * left.var() + float(len(right)) / len(X) * right.var()

In [24]:
class Tree:
    class Node:
        def __init__(self, j =-1,t=-1, y_mean=0, depth=1):
            self.j = j
            self.t = t
            self.y_mean = y_mean
            self.left = None
            self.right = None
            self.depth = depth
    
    def __init__(self, max_depth=0):
        self.left = None
        self.right = None
        self.max_depth = max_depth
        self.root = self.Node()
 
    def _fit(self,node,X,y):  
        if node.depth != self.max_depth and y.var():
            cur_H = 0
            cur_j, cur_t = -1, -1
            for j in range(len(X[0])):
                for t in X[:,j]:
                    left = y[X[:, j] <  t]
                    right = y[X[:, j] >= t]
                    if (len(left) == 0):
                        continue
                    _H = H(X,left,right)
                    if cur_H == 0 or  _H < cur_H :
                        cur_H = _H
                        cur_j = j
                        cur_t = t
            node.j = cur_j
            node.t = cur_t
            left = X[:, node.j] < node.t
            right = X[:, node.j] >= node.t
            if len(X[right]) > 0 and len(X[left]) > 0: 
                node.right = self.Node(depth=node.depth + 1)
                node.left = self.Node(depth=node.depth + 1)
                self._fit(node.left,X[left],y[left])
                self._fit(node.right,X[right],y[right])
        else:
            node.y_mean = y.mean()
            
    def fit(self,X,y):
        self._fit(self.root,X,y)
        
    def split(self,X,j,t,y):
        left = X[:, j] <  t
        right = X[:, j] >= t
        return X[left], y[left], X[right], y[right]
    
    def _predict(self,X, node):
        if node.left is None:
            return node.y_mean
        l, l_ix, r, r_ix = self.split(X, node.j, node.t, np.arange(len(X)))
        y = np.zeros(len(X))
        y[l_ix] = self._predict(l, node.left)
        y[r_ix] = self._predict(r, node.right)
        return y
    def predict(self,X):        
        return self._predict(X, self.root)

In [31]:
tree = Tree(max_depth = 10)

In [32]:
tree.fit(X_train,y_train)

In [33]:
tree.root.right.j

5

In [34]:
answer = tree.predict(X_test)

In [35]:
print answer

[ 36.125       12.6         38.7         14.14        20.26666667  22.1
  23.42857143  19.1         27.9         36.125       16.59        20.26666667
  19.1         29.          22.5         22.66666667  23.25        23.75
  23.81666667  49.5         24.26666667  17.2         12.6         28.7
  16.7         10.2         14.14        41.7         29.8         21.5125
  36.125       23.75        23.42857143  21.7         17.8         14.14
  19.42105263  14.14        23.42857143  23.42857143  20.26666667
  18.78888889  50.          19.42105263  17.2         33.95        20.26666667
  13.25        27.9         13.95        22.1         24.26666667
  24.26666667  16.59        23.42857143  21.5125      14.14        19.1
  21.5125      23.1         27.7         16.59        29.05         7.75
  22.1         16.59        37.6         34.6         33.2         22.1
  31.16666667  16.6         21.5125      10.34166667  21.          18.85
  13.1         20.6         20.5         10.34166667  2

In [36]:
mean_squared_error(y_test, answer)


20.911244192029329